Import all necessary libraries
images taken from: https://archive.stsci.edu/pub/hlsp/hudf12/

In [ ]:
import numpy as np
import sep
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

additional setup for the plot

In [ ]:
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

reading in the input file and making the data usable for the next steps

In [ ]:
image = get_pkg_data_filename('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
data = fits.getdata(image, ext=0)
data = data.byteswap(inplace=True).newbyteorder()

plotting the data

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('fig1.png')

Background subtraction step

In [ ]:
bkg = sep.Background(data)

Global mean and noise of data set

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Making the background a 2d array

In [ ]:
bkg_image = bkg.back()

plotting the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('fig2.png')

plotting background noise

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('fig3.png')

Subtracting the background

In [ ]:
data_sub = data - bkg

Object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

how many objects were detected

In [ ]:
len(objects)

Drawing ellipses around each object on the image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('fig4.png')

listing available fields

In [ ]:
objects.dtype.names

Aperture photometry step

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

showing results of first ten objects

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

Plotting hist of the fluxes

In [ ]:
plt.hist(flux, 100)

Here are the mean, median, and standard deviation of the fluxes. Also a list of the outliers in the data set greater than 30 standard deviations away from the mean, with the only listed outlier being 807.29728357. An outlier greatly messing with the histogram.

In [ ]:
me, medi, standev = np.mean(flux), np.median(flux), np.std(flux)
outliers = (np.abs(flux - me) > 30*standev)
print(me, medi, standev, flux[outliers])

Replotting hist marking the normal standard of deviation in red and the outlier in green. Finding out how many standard deviations away it is.

In [ ]:
plt.hist(flux, 100)
plt.axvline(x=me+standev, ls = "--", color='xkcd:scarlet', alpha=0.7)
plt.axvline(x=807.29728357, ls = "--", color='xkcd:dark green', alpha=0.7)
bah = (807.29728357 - me)/standev
print(bah, "Standard deviations away from the mean")

Now I will attempt to print 3 image on top of each other to create a false color image. Going to import the astropy visualization bits to do it.

In [ ]:
from astropy.visualization import make_lupton_rgb

image1 = get_pkg_data_filename('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')

image2 = get_pkg_data_filename('hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits')

image3 = get_pkg_data_filename('hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits')

bl = fits.open(image1)[0].data
gr = fits.open(image2)[0].data
re = fits.open(image3)[0].data

rgb_default = make_lupton_rgb(re, gr, bl)
plt.imshow(rgb_default, origin='lower')
plt.savefig('fig5.png')